In [20]:
import sys
import os
sys.path.append("../../")
#sys.path.append("workspace/code/Nahush26/Tmet")


In [21]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [22]:
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from trailmet.datasets.classification import DatasetFactory

In [23]:
from trailmet.algorithms.prune.network_slimming_train import NetworkSlimming

In [24]:
data_root = os.getcwd()


In [25]:
train_transform=transforms.Compose([
                                      transforms.Pad(4),
                                      transforms.RandomCrop(32),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                  ])
val_transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                              ])
test_transform = val_transform

In [26]:
transforms1 = {
    'train': train_transform, 
    'val': val_transform, 
    'test': test_transform}
def train_target_transform(label):
    return label

def val_target_transform(label):
    return label

def test_target_transform(label):
    return label

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}
cifar_dataset = DatasetFactory.create_dataset(name = 'CIFAR10', 
                                        root = data_root,
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.2,
                                        transform = transforms1,
                                        target_transform = target_transforms
                                        )

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [27]:
train_loader = torch.utils.data.DataLoader(
        cifar_dataset['train_dataset'], batch_size=64, 
        sampler=cifar_dataset['train_sampler'],
        num_workers=0
    )
val_loader = torch.utils.data.DataLoader(
        cifar_dataset['val_dataset'], batch_size=64, 
        sampler=cifar_dataset['val_sampler'],
        num_workers=0
    )
test_loader = torch.utils.data.DataLoader(
        cifar_dataset['test_dataset'], batch_size=64, 
        sampler=cifar_dataset['test_sampler'],
        num_workers=0
    )
dataloader = {"train" : train_loader , "test" : val_loader}

In [28]:
import yaml
import os
with open(os.path.join(data_root,"cifar10_ns.yml"),'r') as stream:
    data_loaded = yaml.safe_load(stream)

In [29]:
data_loaded

{'NETWORK_SLIMMING': {'GENERAL': {'DATA': 'cifar10',
   'ARCH': 'vgg',
   'NUM_CLASSES': 10,
   'LR': 0.1,
   'MOMENTUM': 0.9,
   'WEIGHT_DECAY': 0.0001,
   'LOG_INTERVAL': 100,
   'DEPTH': 164,
   'OPTIMIZER_NAME': 'SGD',
   'PERCENT': 0.6},
  'SPARSITY_REG': True,
  'THR': 1e-05,
  'FINE_TUNE': False,
  'RESUME': False,
  'TRAIN_BS': 64,
  'TEST_BS': 256,
  'EPOCHS': 100}}

In [30]:
check = NetworkSlimming(data_loaders  = dataloader ,path = './pruned_checkpoint.pth.tar' , save = './pruned_checkpoint.pth.tar', model = 'model_best.pth.tar' , **data_loaded)

In [31]:
check.args.epochs = 1 # for the purpose of checking only
check.args.arch = 'vgg'

In [32]:
model1 = check.train_compress()

vgg(
  (feature): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilatio

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:05<00:00, 31.34it/s, acc=0.423, loss=1.79]


Train set :: Average loss: 1.5353909143447877 


Test set: Average loss: 1.7851692460904456, Accuracy: 42.29 

=> loading checkpoint 'model_best.pth.tar'
=> loaded checkpoint 'model_best.pth.tar' (epoch 1) Prec1: 0.000000
layer index: 3 	 total channel: 64 	 remaining channel: 32
layer index: 6 	 total channel: 64 	 remaining channel: 25
layer index: 10 	 total channel: 128 	 remaining channel: 58
layer index: 13 	 total channel: 128 	 remaining channel: 61
layer index: 17 	 total channel: 256 	 remaining channel: 118
layer index: 20 	 total channel: 256 	 remaining channel: 111
layer index: 23 	 total channel: 256 	 remaining channel: 115
layer index: 26 	 total channel: 256 	 remaining channel: 107
layer index: 30 	 total channel: 512 	 remaining channel: 187
layer index: 33 	 total channel: 512 	 remaining channel: 178
layer index: 36 	 total channel: 512 	 remaining channel: 177
layer index: 39 	 total channel: 512 	 remaining channel: 156
layer index: 43 	 total channel: 512 	 rem

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:04<00:00, 33.33it/s, acc=0.64, loss=0.999]


Train set :: Average loss: 1.2609277143478395 


Test set: Average loss: 0.9993220658818628, Accuracy: 64.03 

